In [24]:
# Cell 1: Imports
import requests
import csv
from typing import Set, List, Dict, Any
import random


In [25]:
# Cell 2: Load Word List
def load_word_list(filename: str = "WORDS.csv") -> Set[str]:
    """
    Load 5-letter words from text file.
    
    Args:
        filename: Path to the file containing words (one word per line)
        
    Returns:
        Set of 5-letter words in uppercase
    """
    words = set()
    
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            for line in file:
                word = line.strip().upper()  # Remove whitespace and convert to uppercase
                if len(word) == 5 and word.isalpha():  # Validate 5-letter alphabetic word
                    words.add(word)
    
    except FileNotFoundError:
        print(f"Error: Could not find file {filename}")
        print("Make sure WORDS.csv is in the same directory as this notebook")
        return set()
    except Exception as e:
        print(f"Error loading word list: {e}")
        return set()
    
    print(f"Loaded {len(words)} words from {filename}")
    return words

# Test the function
word_list = load_word_list()
print(f"Sample words: {sorted(list(word_list))[:10]}")  # Show first 10 words alphabetically


Loaded 3103 words from WORDS.csv
Sample words: ['ABACK', 'ABAFT', 'ABASE', 'ABATE', 'ABBEY', 'ABBOT', 'ABHOR', 'ABIDE', 'ABLER', 'ABODE']


In [26]:
# Cell 3: API Communication Function
def get_feedback(guess: str, seed: int, base_url: str = "https://wordle.votee.dev:8000") -> List[Dict[str, Any]]:
    """
    Submit a guess to the Wordle API and get feedback.
    
    Args:
        guess: 5-letter word guess
        seed: Random seed for consistent game
        base_url: Base URL for the API
        
    Returns:
        List of GuessResult objects with feedback for each letter
    """
    try:
        # Construct the URL for the /random endpoint
        url = f"{base_url}/random"
        
        # Set up parameters
        params = {
            'guess': guess.upper(),
            'seed': seed,
            'size': 5
        }
        
        # Make the API request
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Return the JSON response
        return response.json()
        
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return []
    except Exception as e:
        print(f"Unexpected error: {e}")
        return []

# Test the API function
test_seed = 12345
test_guess = "RAISE"
print(f"Testing API with guess '{test_guess}' and seed {test_seed}")
feedback = get_feedback(test_guess, test_seed)
print(f"Feedback: {feedback}")

# Display feedback in a readable format
if feedback:
    print("\nFormatted feedback:")
    for result in feedback:
        slot = result.get('slot', 'N/A')
        letter = result.get('guess', 'N/A')
        status = result.get('result', 'N/A')
        print(f"Position {slot}: '{letter}' -> {status}")


Testing API with guess 'RAISE' and seed 12345
Feedback: [{'slot': 0, 'guess': 'r', 'result': 'absent'}, {'slot': 1, 'guess': 'a', 'result': 'correct'}, {'slot': 2, 'guess': 'i', 'result': 'absent'}, {'slot': 3, 'guess': 's', 'result': 'absent'}, {'slot': 4, 'guess': 'e', 'result': 'absent'}]

Formatted feedback:
Position 0: 'r' -> absent
Position 1: 'a' -> correct
Position 2: 'i' -> absent
Position 3: 's' -> absent
Position 4: 'e' -> absent


In [27]:
# Cell 4: Enhanced WordleSolver Class with Constraint-Based Logic
class WordleSolver:
    """
    AI solver for Wordle puzzle game using constraint-based letter filtering.
    This approach is robust against unknown words in the target dictionary.
    """
    
    def __init__(self, all_words: Set[str]):
        """
        Initialize the solver with a set of all possible words.
        
        Args:
            all_words: Set of all valid 5-letter words (used for strategic guessing)
        """
        self.all_words = all_words.copy()
        self.guess_count = 0
        self.guesses_made = []
        
        # Constraint-based approach: track what's possible for each position
        self.slot_possibilities = [set('ABCDEFGHIJKLMNOPQRSTUVWXYZ') for _ in range(5)]
        self.must_include_letters = set()  # Letters that must be in the word somewhere
        self.absent_letters = set()        # Letters that are definitely not in the word
        self.confirmed_positions = {}      # slot -> letter for confirmed positions
        
    def get_next_guess(self) -> str:
        """
        Get the next word to guess using constraint-based logic.
        
        Returns:
            Next word to guess
        """
        if self.guess_count == 0:
            # First guess: use a word with high vowel count and common letters
            first_guess = "RAISE"  # R, A, I, S, E are common letters with good vowel coverage
            return first_guess
        
        # Filter our word list based on current constraints
        valid_words = self._filter_words_by_constraints()
        
        if valid_words:
            # Strategy: Pick the word that uses the most uncommon letters we haven't tried yet
            # For now, just pick the first valid word (can be enhanced later)
            return next(iter(valid_words))
        else:
            # Fallback: Generate a word that satisfies our constraints
            # This is the key innovation - we don't give up when our word list is exhausted
            print("🔧 No words in dictionary match constraints. Generating failsafe guess...")
            return self._generate_constraint_satisfying_word()
    
    def _filter_words_by_constraints(self) -> Set[str]:
        """
        Filter the word list based on current constraints.
        
        Returns:
            Set of words that satisfy all current constraints
        """
        valid_words = set()
        
        for word in self.all_words:
            if self._word_satisfies_constraints(word):
                valid_words.add(word)
        
        return valid_words
    
    def _word_satisfies_constraints(self, word: str) -> bool:
        """
        Check if a word satisfies all current constraints.
        
        Args:
            word: Word to check
            
        Returns:
            True if word satisfies all constraints
        """
        word = word.upper()
        
        # Check confirmed positions
        for slot, letter in self.confirmed_positions.items():
            if word[slot] != letter:
                return False
        
        # Check slot possibilities
        for slot, letter in enumerate(word):
            if letter not in self.slot_possibilities[slot]:
                return False
        
        # Check that all required letters are present
        for letter in self.must_include_letters:
            if letter not in word:
                return False
        
        # Check that no absent letters are present
        for letter in self.absent_letters:
            if letter in word:
                return False
        
        return True
    
    def _generate_constraint_satisfying_word(self) -> str:
        """
        Generate a word that satisfies all current constraints.
        This is our failsafe when no dictionary words work.
        
        Returns:
            A 5-letter word that satisfies constraints
        """
        word = [''] * 5
        
        # First, fill in confirmed positions
        for slot, letter in self.confirmed_positions.items():
            word[slot] = letter
        
        # Then, place required letters that don't have confirmed positions
        unplaced_required = self.must_include_letters - set(self.confirmed_positions.values())
        
        for letter in unplaced_required:
            # Find a slot where this letter can go
            for slot in range(5):
                if word[slot] == '' and letter in self.slot_possibilities[slot]:
                    word[slot] = letter
                    break
        
        # Fill remaining slots with valid letters
        for slot in range(5):
            if word[slot] == '':
                # Choose the first valid letter for this slot
                available_letters = (self.slot_possibilities[slot] - 
                                   self.absent_letters - 
                                   set(word))  # Avoid duplicates
                if available_letters:
                    word[slot] = next(iter(available_letters))
                else:
                    # Last resort: use any letter that's possible for this slot
                    word[slot] = next(iter(self.slot_possibilities[slot]))
        
        return ''.join(word)
    
    def update_constraints(self, guess: str, feedback: List[Dict[str, Any]]):
        """
        Update constraints based on feedback from a guess.
        
        Args:
            guess: The word that was guessed
            feedback: List of feedback objects from the API
        """
        guess = guess.upper()
        self.guess_count += 1
        self.guesses_made.append(guess)
        
        print(f"\n🔍 Processing guess '{guess}' with feedback:")
        for result in feedback:
            print(f"  Position {result['slot']}: '{result['guess'].upper()}' -> {result['result']}")
        
        # Track letters by their feedback in this guess
        correct_letters = {}    # slot -> letter
        present_letters = {}    # slot -> letter (letter is in word but wrong position)
        absent_letters = set()  # letters not in word
        
        # First pass: categorize feedback
        for result in feedback:
            slot = result['slot']
            letter = result['guess'].upper()
            status = result['result']
            
            if status == 'correct':
                correct_letters[slot] = letter
            elif status == 'present':
                present_letters[slot] = letter
            elif status == 'absent':
                absent_letters.add(letter)
        
        # Second pass: update constraints carefully
        # Handle correct positions
        for slot, letter in correct_letters.items():
            self.confirmed_positions[slot] = letter
            self.slot_possibilities[slot] = {letter}  # Only this letter is possible
            self.must_include_letters.add(letter)
        
        # Handle present letters (in word but wrong position)
        for slot, letter in present_letters.items():
            self.must_include_letters.add(letter)
            self.slot_possibilities[slot].discard(letter)  # Not in this position
        
        # Handle absent letters (tricky with duplicates)
        for letter in absent_letters:
            # Only mark as truly absent if it's not also correct or present
            if (letter not in correct_letters.values() and 
                letter not in present_letters.values()):
                self.absent_letters.add(letter)
                # Remove from all slot possibilities
                for slot in range(5):
                    self.slot_possibilities[slot].discard(letter)
        
        # Debug output
        print(f"🔍 Updated constraints:")
        print(f"  Confirmed positions: {self.confirmed_positions}")
        print(f"  Must include letters: {self.must_include_letters}")
        print(f"  Absent letters: {self.absent_letters}")
        print(f"  Slot possibilities: {[sorted(list(s)) for s in self.slot_possibilities]}")
        
        # Count valid words
        valid_words = self._filter_words_by_constraints()
        print(f"After guess '{guess}': {len(valid_words)} dictionary words match constraints")
        
        if len(valid_words) <= 10 and len(valid_words) > 0:
            print(f"Matching words: {sorted(list(valid_words))}")
    
    # Keep the old method name for compatibility with the game runner
    def update_possible_words(self, guess: str, feedback: List[Dict[str, Any]]):
        """Compatibility wrapper for the old method name."""
        self.update_constraints(guess, feedback)
    
    @property
    def possible_words(self) -> Set[str]:
        """Compatibility property - returns words that match constraints."""
        return self._filter_words_by_constraints()

# Test the enhanced solver class
print("Testing Enhanced WordleSolver class...")
solver = WordleSolver(word_list)
print(f"Initialized solver with {len(solver.all_words)} words in dictionary")
print(f"First guess: {solver.get_next_guess()}")
print(f"Initial constraints allow {len(solver.possible_words)} words")


Testing Enhanced WordleSolver class...
Initialized solver with 3103 words in dictionary
First guess: RAISE
Initial constraints allow 3103 words


In [28]:
# Cell 5: The Game Runner
def play_wordle_game(seed: int = None, max_attempts: int = 6) -> bool:
    """
    Play a complete Wordle game using the AI solver.
    
    Args:
        seed: Random seed for the game (if None, will use a random seed)
        max_attempts: Maximum number of guesses allowed
        
    Returns:
        True if the game was won, False otherwise
    """
    # Configuration
    base_url = "https://wordle.votee.dev:8000"
    
    # Use random seed if not provided
    if seed is None:
        seed = random.randint(1, 100000)
    
    print(f"🎮 Starting Wordle Game with seed: {seed}")
    print("=" * 50)
    
    # Load word list and create solver
    words = load_word_list()
    if not words:
        print("❌ Failed to load word list!")
        return False
    
    solver = WordleSolver(words)
    
    # Game loop
    for attempt in range(1, max_attempts + 1):
        print(f"\n🔄 Attempt {attempt}/{max_attempts}")
        
        # Get next guess from solver
        guess = solver.get_next_guess()
        print(f"🤖 AI Guess: {guess}")
        
        # Submit guess to API
        feedback = get_feedback(guess, seed, base_url)
        
        if not feedback:
            print("❌ Failed to get feedback from API!")
            return False
        
        # Display feedback
        print("📋 Feedback:")
        all_correct = True
        for result in feedback:
            slot = result['slot']
            letter = result['guess'].upper()
            status = result['result']
            
            # Choose emoji based on status
            if status == 'correct':
                emoji = "🟢"
            elif status == 'present':
                emoji = "🟡"
                all_correct = False
            else:  # absent
                emoji = "⚫"
                all_correct = False
            
            print(f"  Position {slot}: '{letter}' {emoji} ({status})")
        
        # Check if we won
        if all_correct:
            print(f"\n🎉 SUCCESS! Won in {attempt} attempts!")
            print(f"🏆 The word was: {guess}")
            return True
        
        # Update solver with feedback for next iteration
        solver.update_possible_words(guess, feedback)
        
        # Check if we have possible words left
        if not solver.possible_words:
            print("❌ No possible words remaining! Something went wrong.")
            return False
    
    print(f"\n😞 Game Over! Failed to solve in {max_attempts} attempts.")
    print(f"🤔 Remaining possibilities: {len(solver.possible_words)}")
    if solver.possible_words and len(solver.possible_words) <= 5:
        print(f"🔍 Remaining words: {sorted(list(solver.possible_words))}")
    
    return False

# Run the game!
print("🚀 Starting AI Wordle Solver...")
print("This will play a complete game against the API")
print()

# Play a game with a fixed seed for reproducibility
game_seed = 50
success = play_wordle_game(seed=game_seed)

if success:
    print("\n✅ AI successfully solved the Wordle!")
else:
    print("\n❌ AI failed to solve the Wordle this time.")

print("\n🔄 You can run this cell again to play another game!")


🚀 Starting AI Wordle Solver...
This will play a complete game against the API

🎮 Starting Wordle Game with seed: 50
Loaded 3103 words from WORDS.csv

🔄 Attempt 1/6
🤖 AI Guess: RAISE
📋 Feedback:
  Position 0: 'R' 🟡 (present)
  Position 1: 'A' ⚫ (absent)
  Position 2: 'I' ⚫ (absent)
  Position 3: 'S' ⚫ (absent)
  Position 4: 'E' 🟡 (present)

🔍 Processing guess 'RAISE' with feedback:
  Position 0: 'R' -> present
  Position 1: 'A' -> absent
  Position 2: 'I' -> absent
  Position 3: 'S' -> absent
  Position 4: 'E' -> present
🔍 Updated constraints:
  Confirmed positions: {}
  Must include letters: {'R', 'E'}
  Absent letters: {'I', 'S', 'A'}
  Slot possibilities: [['B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'T', 'U', 'V', 'W', 'X'

In [29]:
# Cell 7: Debugging and Analysis Tools
def analyze_mystery_word(all_guesses_and_feedback: List[tuple], word_list: Set[str]) -> None:
    """
    Analyze what the mystery word could be based on all feedback received.
    
    Args:
        all_guesses_and_feedback: List of (guess, feedback) tuples
        word_list: Set of all possible words
    """
    print("🔍 MYSTERY WORD ANALYSIS")
    print("=" * 50)
    
    # Build constraints from all feedback
    correct_positions = {}
    present_letters = set()
    absent_letters = set()
    
    for guess, feedback in all_guesses_and_feedback:
        print(f"\nGuess: {guess}")
        for result in feedback:
            slot = result['slot']
            letter = result['guess'].upper()
            status = result['result']
            
            if status == 'correct':
                correct_positions[slot] = letter
                print(f"  Position {slot}: {letter} ✓ (correct)")
            elif status == 'present':
                present_letters.add(letter)
                print(f"  Position {slot}: {letter} ~ (present elsewhere)")
            else:
                absent_letters.add(letter)
                print(f"  Position {slot}: {letter} ✗ (absent)")
    
    print(f"\n📋 FINAL CONSTRAINTS:")
    print(f"  Correct positions: {correct_positions}")
    print(f"  Must contain: {present_letters}")
    print(f"  Must not contain: {absent_letters}")
    
    # Find all words that match these constraints
    candidates = set()
    
    for word in word_list:
        matches = True
        
        # Check correct positions
        for pos, letter in correct_positions.items():
            if word[pos] != letter:
                matches = False
                break
        
        if not matches:
            continue
        
        # Check present letters
        for letter in present_letters:
            if letter not in word:
                matches = False
                break
            # Make sure present letters are not in positions where they were guessed
            for guess, feedback in all_guesses_and_feedback:
                for result in feedback:
                    if (result['guess'].upper() == letter and 
                        result['result'] == 'present' and 
                        word[result['slot']] == letter):
                        matches = False
                        break
                if not matches:
                    break
        
        if not matches:
            continue
        
        # Check absent letters
        for letter in absent_letters:
            if letter in word and letter not in present_letters and letter not in correct_positions.values():
                matches = False
                break
        
        if matches:
            candidates.add(word)
    
    print(f"\n🎯 CANDIDATE WORDS: {len(candidates)}")
    if candidates:
        for word in sorted(candidates):
            print(f"  {word}")
    else:
        print("  No words found! This suggests:")
        print("  1. The mystery word is not in our word list")
        print("  2. There's a bug in our constraint logic")
        print("  3. The API feedback has inconsistencies")

def test_specific_seed(seed: int) -> None:
    """Test a specific seed with detailed debugging"""
    print(f"🧪 TESTING SEED {seed} WITH FULL DEBUG")
    print("=" * 60)
    
    # Store all guesses and feedback for analysis
    all_guesses_feedback = []
    
    base_url = "https://wordle.votee.dev:8000"
    words = load_word_list()
    solver = WordleSolver(words)
    
    for attempt in range(1, 7):
        print(f"\n🔄 Attempt {attempt}/6")
        guess = solver.get_next_guess()
        print(f"🤖 AI Guess: {guess}")
        
        feedback = get_feedback(guess, seed, base_url)
        if not feedback:
            print("❌ Failed to get feedback!")
            break
        
        # Store for analysis
        all_guesses_feedback.append((guess, feedback))
        
        # Check if won
        all_correct = all(result['result'] == 'correct' for result in feedback)
        if all_correct:
            print(f"🎉 SUCCESS! The word was: {guess}")
            return
        
        # Update solver with debugging
        solver.update_possible_words(guess, feedback)
        
        if not solver.possible_words:
            print("\n❌ No possible words remaining!")
            break
    
    # If we get here, we failed - analyze why
    print("\n" + "=" * 60)
    print("FAILURE ANALYSIS")
    print("=" * 60)
    
    analyze_mystery_word(all_guesses_feedback, words)

# Test the problematic seed
print("🚀 Ready to debug! Run test_specific_seed(50) to analyze the failing case.")


🚀 Ready to debug! Run test_specific_seed(50) to analyze the failing case.


In [30]:
# Cell 6: Testing and Experimentation
def run_multiple_games(num_games: int = 5) -> Dict[str, Any]:
    """
    Run multiple games to test the solver's performance.
    
    Args:
        num_games: Number of games to run
        
    Returns:
        Dictionary with performance statistics
    """
    results = {
        'games_won': 0,
        'games_lost': 0,
        'total_attempts': 0,
        'attempt_distribution': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0},
        'seeds_used': []
    }
    
    print(f"🎯 Running {num_games} games to test solver performance...")
    print("=" * 60)
    
    for game_num in range(1, num_games + 1):
        seed = random.randint(1, 100000)
        results['seeds_used'].append(seed)
        
        print(f"\n🎮 Game {game_num}/{num_games} (Seed: {seed})")
        print("-" * 30)
        
        # Run game with minimal output
        base_url = "https://wordle.votee.dev:8000"
        words = load_word_list()
        solver = WordleSolver(words)
        
        won = False
        attempts_used = 0
        
        for attempt in range(1, 7):
            attempts_used = attempt
            guess = solver.get_next_guess()
            feedback = get_feedback(guess, seed, base_url)
            
            if not feedback:
                print(f"❌ API Error in game {game_num}")
                break
            
            # Check if won
            all_correct = all(result['result'] == 'correct' for result in feedback)
            
            if all_correct:
                won = True
                print(f"✅ Won in {attempt} attempts! Word: {guess}")
                results['games_won'] += 1
                results['attempt_distribution'][attempt] += 1
                break
            
            # Update solver
            solver.update_possible_words(guess, feedback)
            
            if not solver.possible_words:
                print(f"❌ No possible words remaining in game {game_num}")
                break
        
        if not won:
            print(f"❌ Lost game {game_num}")
            results['games_lost'] += 1
        
        results['total_attempts'] += attempts_used
    
    # Calculate and display statistics
    print("\n" + "=" * 60)
    print("📊 PERFORMANCE SUMMARY")
    print("=" * 60)
    
    win_rate = (results['games_won'] / num_games) * 100
    avg_attempts = results['total_attempts'] / num_games
    
    print(f"🏆 Games Won: {results['games_won']}/{num_games} ({win_rate:.1f}%)")
    print(f"💔 Games Lost: {results['games_lost']}/{num_games}")
    print(f"📈 Average Attempts: {avg_attempts:.1f}")
    
    print(f"\n📋 Attempt Distribution:")
    for attempts, count in results['attempt_distribution'].items():
        if count > 0:
            percentage = (count / results['games_won']) * 100 if results['games_won'] > 0 else 0
            print(f"  {attempts} attempts: {count} games ({percentage:.1f}%)")
    
    return results

# Uncomment the line below to run performance testing
performance_stats = run_multiple_games(20)

print("🧪 Experimental cell ready!")
print("Uncomment the line above to run multiple games and see performance statistics.")


🎯 Running 20 games to test solver performance...

🎮 Game 1/20 (Seed: 63660)
------------------------------
Loaded 3103 words from WORDS.csv

🔍 Processing guess 'RAISE' with feedback:
  Position 0: 'R' -> absent
  Position 1: 'A' -> correct
  Position 2: 'I' -> absent
  Position 3: 'S' -> present
  Position 4: 'E' -> absent
🔍 Updated constraints:
  Confirmed positions: {1: 'A'}
  Must include letters: {'S', 'A'}
  Absent letters: {'R', 'I', 'E'}
  Slot possibilities: [['A', 'B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], ['A'], ['A', 'B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], ['A', 'B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], ['A', 'B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']]
After guess 'RAISE': 80 dictionary words match constraints